In [8]:
import os
import random
import shutil

In [ ]:
def keep_only_class0(data_dir):
    label_dir = data_dir + '/labels'
    image_dir = data_dir + '/images'
    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            label_path = os.path.join(label_dir, file)
            image_path = os.path.join(image_dir, file).replace('.txt', '.jpg')
            with open(label_path, 'r') as f:
                lines = f.readlines()
            lines = [l for l in lines if l.startswith("0 ")]
            if lines:
                with open(label_path, 'w') as f:
                    f.writelines(lines)
            else:
                os.remove(label_path)  # Xoá nhãn nếu không còn fire
                os.remove(image_path)  # Xoá ảnh nếu không còn fire

# Lọc toàn bộ
for split in ["train"]:
    keep_only_class0(f"data/FireDetectionYOLO/{split}")


In [9]:
def split_dataset(img_dir, label_dir, output_dir, val_ratio=0.1, test_ratio=0.1, seed=42):
    random.seed(seed)

    # Lấy danh sách ảnh
    images = [f for f in os.listdir(img_dir) if f.endswith((".jpg", ".png"))]
    random.shuffle(images)

    total = len(images)
    test_count = int(total * test_ratio)
    val_count = int(total * val_ratio)

    test_imgs = images[:test_count]
    val_imgs = images[test_count:test_count + val_count]
    train_imgs = images[test_count + val_count:]

    print(f"Total: {total} → Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    def copy_files(img_list, split):
        img_out = os.path.join(output_dir, split, "images")
        lbl_out = os.path.join(output_dir, split, "labels")
        os.makedirs(img_out, exist_ok=True)
        os.makedirs(lbl_out, exist_ok=True)

        for img_name in img_list:
            base = os.path.splitext(img_name)[0]
            shutil.copy(os.path.join(img_dir, img_name), os.path.join(img_out, img_name))

            label_file = base + ".txt"
            src_lbl = os.path.join(label_dir, label_file)
            if os.path.exists(src_lbl):
                shutil.copy(src_lbl, os.path.join(lbl_out, label_file))

    copy_files(train_imgs, "train")
    copy_files(val_imgs, "val")
    copy_files(test_imgs, "test")

split_dataset(
    img_dir="data/FireDetection/train/images",
    label_dir="data/FireDetection/train/labels",
    output_dir="data/FireDetection/splited",
    val_ratio=0.1,
    test_ratio=0.1
)


Total: 7704 → Train: 6164, Val: 770, Test: 770
